In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/home/watson_chua/efs/data/aibots/AIBOTS_CONVERSATIONS/AIBOTS_ANALYTICS/PLAYGROUND.csv")

In [ ]:
df.head()

In [ ]:
import sys
sys.path.append("..")

In [ ]:
!pwd

In [ ]:
grouped = df.groupby("conversationId")

In [ ]:
len(grouped)

In [ ]:
from pathlib import Path
output_folder_path = Path("../kura/data/aibots_conversations")
output_folder_path.mkdir(parents=True, exist_ok=True)

In [ ]:
from uuid import uuid4
from kura.types import Conversation, Message
import json
from datetime import datetime

all_conversations = []
sample_size = 1000
count = 0
for convo_id, group in grouped:

    messages = []
    
    for _, row in group.iterrows():
        user_message = Message(
                created_at=row["queryDatetime"],
                role="user",
                content=row["query"],
                id=row["_id"],
                )
        
        assistant_message = Message(
                created_at=row["queryDatetime"],
                role="assistant",
                content=row["response"],
                id=row["_id"],
                )
                            
        messages.append(user_message)
        messages.append(assistant_message)

    
    current_convo = Conversation(
        messages=messages,
        chat_id=convo_id,
        created_at=datetime.now(),
    )
        
    all_conversations.append(current_convo)
    count +=1
    if count > sample_size:
        break
    


In [ ]:
# from kura.kura.embedding import OpenAIEmbeddingModel
# from kura.kura.kura import Kura
# from kura.kura.summarisation import SummaryModel
# from kura.kura.dimensionality import DimensionalityReduction
# from umap import UMAP


# kura = Kura(
#     embedding_model=OpenAIEmbeddingModel(),
#     summarisation_model=SummaryModel(),
#     dimensionality_reduction=DimensionalityReduction(),
#     max_clusters=10,
#     checkpoint_dir="./checkpoints",
# )



In [ ]:
%pip install umap-learn
from umap import UMAP

In [ ]:
run(kura.cluster_conversations(all_conversations))

In [ ]:
from dotenv import load_dotenv, find_dotenv
from openai import AsyncOpenAI


load_dotenv(find_dotenv())

In [ ]:
import instructor
import os
from openai import AsyncOpenAI


client = instructor.from_openai(
            AsyncOpenAI(api_key=os.environ["OPENAI_API_KEY"], base_url=os.environ["OPENAI_API_BASE"]),
            # use_async=True,
            # mode=instructor.Mode.GEMINI_JSON,
        )

In [ ]:
from pydantic import BaseModel


class ConversationSummary(BaseModel):
    chat_id: str
    summary: str
    metadata: dict


class GeneratedSummary(BaseModel):
    summary: str
    
client.chat.completions.create(
            model = "gemini-1.5-flash-001",
            messages=[{"role": "user", "content": "hello"}],
            response_model=GeneratedSummary,
)

In [ ]:
resp = await client.chat.completions.create(
            model = "gpt-4o-eastus",
            # model = "gemini-1.5-flash-001_lb",
            messages=[
                {
                    "role": "system",
                    "content": """
                    Generate a summary of the task that the user is asking the language model to do based off the following conversation.


                    The summary should be concise and short. It should be at most 1-2 sentences and at most 30 words. Here are some examples of summaries:
                    - The user's overall request for the assistant is to help implementing a React component to display a paginated list of users from a database.
                    - The user's overall request for the assistant is to debug a memory leak in their Python data processing pipeline.
                    - The user's overall request for the assistant is to design and architect a REST API for a social media application.
                    

                    Here is the conversation
                    <messages>
                    {% for message in messages %}
                        <message>{{message.role}}: {{message.content}} </message>
                    {% endfor %}
                    </messages>

                    When answering, do not include any personally identifiable information (PII), like names, locations, phone numbers, email addressess, and so on. When answering, do not include any proper nouns. Make sure that you're clear, concise and that you get to the point in at most two sentences.
                    
                    For example:

                    Remember that
                    - Summaries should be concise and short. They should each be at most 1-2 sentences and at most 30 words.
                    - Summaries should start with "The user's overall request for the assistant is to"
                    - Make sure to omit any personally identifiable information (PII), like names, locations, phone numbers, email addressess, company names and so on.
                    - Make sure to indicate specific details such as programming languages, frameworks, libraries and so on which are relevant to the task.
                    """,
                }
            ],
            # context={"messages": conversation.messages},
            context={"messages": all_conversations[1].messages},
            response_model=GeneratedSummary,
        )

In [ ]:
resp.summary

In [ ]:
ConversationSummary(
            chat_id=conversation.chat_id,
            summary=resp.summary,
            metadata={
                "conversation_turns": len(conversation.messages),
            }